In [1]:
from pymatgen.ext.matproj import MPRester
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from jupyter_jsmol.pymatgen import quick_view
import numpy as np

In [2]:
#Téléchargement du fichier CIF
API_KEY = "903fk7jExqhMBNCB46b"
MP = "mp-862947"
mpr = MPRester(API_KEY)
structure = mpr.get_structure_by_material_id(MP, conventional_unit_cell = True)
structureprimitive = SpacegroupAnalyzer(structure).get_primitive_standard_structure()
structure.to(filename="mp-862947.cif")

In [3]:
#Visualiser la structure du matériau
view = quick_view(structure)
view.script("set antialiasDisplay ON")
view.script("set zoomLarge false")
view.script("set cameraDepth 60")
display(view)

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [4]:
#Visualiser la structure primitive du matériau
view = quick_view(structureprimitive)
view.script("set antialiasDisplay ON")
view.script("set zoomLarge false")
view.script("set cameraDepth 60")
display(view)

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [5]:
#Position de 3 atomes différents
atom1 = {"element": "Bi", "x": 0.0, "y": 0.0, "z": 0.0}
atom2 = {"element": "Ba", "x": 0.25, "y": 0.25, "z": 0.75}
atom3 = {"element": "Au", "x": 0.5, "y": 0.0, "z": 0.0}
atoms = [atom1, atom2, atom3]

# éléments de symétrie
miroirxy = np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]])  
miroirxz = np.array([[-1, 0, 0], [0, 1, 0], [0, 0, -1]])  
rotationz = np.array([[0, 0, 1], [0, 1, 0], [-1, 0, 0]])  

# Liste de matrices de symétrie
symmetries = [miroirxy, miroirxz, rotationz]

In [6]:
def apply_and_display_symmetry(atom, symmetry, view):
    # Récupérer les coordonnées de l'atome initial
    x, y, z = atom["x"], atom["y"], atom["z"]
    
    # Appliquer la rotation à l'atome
    coords = np.array([[x], [y], [z]])
    new_coords = np.dot(symmetry, coords)
    new_x, new_y, new_z = new_coords.ravel().tolist()
    
    # Ajouter une sphère pour représenter le nouvel atome
    view.script(f'spacefill @{new_x:.2f} @{new_y:.2f} @{new_z:.2f} color skyblue')
    
    # Ajouter une ligne reliant l'atome initial au nouvel atome
    view.script(f'wireframe 0.2 {{( @{x:.2f} @{y:.2f} @{z:.2f} )}} {{( @{new_x:.2f} @{new_y:.2f} @{new_z:.2f} )}} color {atom["element"]}')
    
    # Ajouter une étiquette pour le nouvel atome
    view.script(f'label {atom["element"]} {new_x:.2f} {new_y:.2f} {new_z:.2f} "{atom["element"]}{new_x:.2f}{new_y:.2f}{new_z:.2f}"')
    return {"element": atom["element"], "x": new_x, "y": new_y, "z": new_z}


In [7]:
new_atoms = []
for i, atom in enumerate(atoms):
    for j, symmetry in enumerate(symmetries):
        new_atom = apply_and_display_symmetry(atom, symmetry, view)
        new_atoms.append(new_atom)
        # ajouter une flèche pour visualiser l'effet de symétrie
        view.script(f"draw symm {i*3+j+1} arrow {{0 0 0}} {{1 1 1}}") 
print (atoms)
print("NEW")
print (new_atoms)

[{'element': 'Bi', 'x': 0.0, 'y': 0.0, 'z': 0.0}, {'element': 'Ba', 'x': 0.25, 'y': 0.25, 'z': 0.75}, {'element': 'Au', 'x': 0.5, 'y': 0.0, 'z': 0.0}]
NEW
[{'element': 'Bi', 'x': 0.0, 'y': 0.0, 'z': 0.0}, {'element': 'Bi', 'x': 0.0, 'y': 0.0, 'z': 0.0}, {'element': 'Bi', 'x': 0.0, 'y': 0.0, 'z': 0.0}, {'element': 'Ba', 'x': 0.25, 'y': -0.25, 'z': -0.75}, {'element': 'Ba', 'x': -0.25, 'y': 0.25, 'z': -0.75}, {'element': 'Ba', 'x': 0.75, 'y': 0.25, 'z': -0.25}, {'element': 'Au', 'x': 0.5, 'y': 0.0, 'z': 0.0}, {'element': 'Au', 'x': -0.5, 'y': 0.0, 'z': 0.0}, {'element': 'Au', 'x': 0.0, 'y': 0.0, 'z': -0.5}]


In [8]:
# afficher les nouveaux atomes
for i, atom in enumerate(new_atoms):
    x, y, z = atom["x"], atom["y"], atom["z"]
    view.script(f'label {atom["element"]} {x:.2f} {y:.2f} {z:.2f} "{atom["element"]}{x:.2f}{y:.2f}{z:.2f}"')
    view.script(f'spacefill {atom["element"]} @{x:.2f} @{y:.2f} @{z:.2f}')
    
display(view)

JsmolView(layout=Layout(align_self='stretch', height='400px'))